In [2]:
import math
import collections

import pandas as pd

from keras import regularizers
from keras.models import Model, Sequential
from keras.layers import *
from keras.wrappers.scikit_learn import KerasClassifier

import params
from utils.sequence_data import data_to_sequences_and_labels
from utils.metrics import print_report_for_binary_classfier
from utils.preprocessing import probs_to_binary_classes
from utils.hyperparams import hyperparams_search
from utils.plot import plot_train_validation_metric

Using TensorFlow backend.


In [3]:
dataset = pd.read_csv("../datasets/data_for_models/dataset_1996-01-01_2019-08-22.csv",
                           header=0, parse_dates=[0], index_col=0)

INPUT_SHAPE = (params.LOOKBACK//params.STEP, dataset.shape[-1] )

In [4]:
#params for generating sequences 
train_max_idx = math.ceil(len(dataset)*params.TRAIN_RATIO)
val_max_idx = math.ceil(len(dataset)*(params.TRAIN_RATIO+params.VAL_RATIO))
label_index = len(dataset.columns) -1


# prepare data
train_X, train_Y = data_to_sequences_and_labels(dataset.to_numpy(), params.LOOKBACK, 
                                                params.STEP, 
                                                0, train_max_idx, 
                                                params.DELAY,
                                                label_index) 
val_X, val_Y = data_to_sequences_and_labels(dataset.to_numpy(), params.LOOKBACK, 
                                                params.STEP, 
                                                train_max_idx+1, val_max_idx, 
                                                params.DELAY,
                                                label_index)
test_X, test_Y = data_to_sequences_and_labels(dataset.to_numpy(), params.LOOKBACK, 
                                                params.STEP, 
                                                val_max_idx+1, None, 
                                                params.DELAY,
                                                label_index)
X, Y = data_to_sequences_and_labels(dataset.to_numpy(), params.LOOKBACK,
                                   params.STEP,
                                   0, val_max_idx,
                                   params.DELAY
                                   ,label_index)

In [5]:
def lstm_model(input_shape, hidden_unit=32, n_layer=2, l2_weight=0.01):
    model = Sequential()
    is_return_sequences = True if n_layer>=2 else False
    
    # first layer
    model.add(LSTM(hidden_unit, 
                   input_shape=input_shape,
                   kernel_regularizer=regularizers.l2(l2_weight),
                   recurrent_regularizer=regularizers.l2(l2_weight),
                   return_sequences=is_return_sequences))
    
    # intermediate layers
    if n_layer >= 3:
        for _ in range(n_layer-2):
            model.add(LSTM(hidden_unit,
                           kernel_regularizer=regularizers.l2(l2_weight),
                           recurrent_regularizer=regularizers.l2(l2_weight),
                           return_sequences=is_return_sequences))

    
    # last layer
    if n_layer >= 2:
        model.add(LSTM(hidden_unit,
                       recurrent_regularizer=regularizers.l2(l2_weight),
                       kernel_regularizer=regularizers.l2(l2_weight)))
    
    model.add(Dense(1, activation='sigmoid'))
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

In [8]:
#Prepare for gridsearchCV
from sklearn.metrics import matthews_corrcoef, make_scorer
from sklearn.model_selection import TimeSeriesSplit, GridSearchCV 
cv = TimeSeriesSplit(n_splits=2)
matthews_score = make_scorer(matthews_corrcoef, greater_is_better=True)


In [31]:
lstm_params = dict(hidden_unit=[48, 64], 
                 n_layer=[3,4], 
                 epochs=[100, 250],
                 l2_weight=[0.001, 0.01])

# lstm_params = dict(hidden_unit=[64], 
#                  n_layer=[3], 
#                  epochs=[100],
#                  l2_weight=[0])

lstm_clf = GridSearchCV(estimator=KerasClassifier(build_fn=lstm_model, input_shape=INPUT_SHAPE, verbose=1),
                      param_grid=lstm_params,
                      cv =cv,
                      scoring=matthews_score)
grid_result=lstm_clf.fit(X,Y)

Epoch 1/100
1634/1634 [==============================] - 7s 4ms/step - loss: 0.8832 - acc: 0.6769
Epoch 2/100
1634/1634 [==============================] - 2s 1ms/step - loss: 0.7340 - acc: 0.6769
Epoch 3/100
1634/1634 [==============================] - 2s 1ms/step - loss: 0.6563 - acc: 0.6830
Epoch 4/100
1634/1634 [==============================] - 2s 1ms/step - loss: 0.6109 - acc: 0.7118
Epoch 5/100
1634/1634 [==============================] - 2s 1ms/step - loss: 0.6056 - acc: 0.6928
Epoch 6/100
1634/1634 [==============================] - 2s 1ms/step - loss: 0.5803 - acc: 0.7087
Epoch 7/100
1634/1634 [==============================] - 2s 1ms/step - loss: 0.5574 - acc: 0.7289
Epoch 8/100
1634/1634 [==============================] - 2s 1ms/step - loss: 0.5594 - acc: 0.7313
Epoch 9/100
1634/1634 [==============================] - 2s 1ms/step - loss: 0.5661 - acc: 0.7160
Epoch 10/100
1634/1634 [==============================] - 2s 1ms/step - loss: 0.5474 - acc: 0.7399
Epoch 11/100
1634/1

1634/1634 [==============================] - 2s 1ms/step - loss: 0.2393 - acc: 0.9284
Epoch 84/100
1634/1634 [==============================] - 2s 1ms/step - loss: 0.2525 - acc: 0.9186
Epoch 85/100
1634/1634 [==============================] - 2s 1ms/step - loss: 0.2300 - acc: 0.9315
Epoch 86/100
1634/1634 [==============================] - 2s 1ms/step - loss: 0.2349 - acc: 0.9327
Epoch 87/100
1634/1634 [==============================] - 2s 1ms/step - loss: 0.2127 - acc: 0.9394
Epoch 88/100
1634/1634 [==============================] - 2s 1ms/step - loss: 0.2336 - acc: 0.9345
Epoch 89/100
1634/1634 [==============================] - 2s 1ms/step - loss: 0.2410 - acc: 0.9253
Epoch 90/100
1634/1634 [==============================] - 2s 1ms/step - loss: 0.1846 - acc: 0.9547
Epoch 91/100
1634/1634 [==============================] - 2s 1ms/step - loss: 0.2454 - acc: 0.9204
Epoch 92/100
1634/1634 [==============================] - 2s 1ms/step - loss: 0.1908 - acc: 0.9480
Epoch 93/100
1634/1634 

3267/3267 [==============================] - 3s 887us/step - loss: 0.2708 - acc: 0.9039
Epoch 65/100
3267/3267 [==============================] - 3s 887us/step - loss: 0.2638 - acc: 0.9063
Epoch 66/100
3267/3267 [==============================] - 3s 883us/step - loss: 0.2621 - acc: 0.9091
Epoch 67/100
3267/3267 [==============================] - 3s 882us/step - loss: 0.3105 - acc: 0.8871
Epoch 68/100
3267/3267 [==============================] - 3s 887us/step - loss: 0.2625 - acc: 0.9115
Epoch 69/100
3267/3267 [==============================] - 3s 879us/step - loss: 0.2592 - acc: 0.9140
Epoch 70/100
3267/3267 [==============================] - 3s 880us/step - loss: 0.2680 - acc: 0.9042
Epoch 71/100
3267/3267 [==============================] - 3s 881us/step - loss: 0.2529 - acc: 0.9128
Epoch 72/100
3267/3267 [==============================] - 3s 888us/step - loss: 0.2445 - acc: 0.9164
Epoch 73/100
3267/3267 [==============================] - 3s 891us/step - loss: 0.2539 - acc: 0.9094
Epo

1634/1634 [==============================] - 2s 1ms/step - loss: 0.4900 - acc: 0.7785
Epoch 46/100
1634/1634 [==============================] - 2s 1ms/step - loss: 0.4928 - acc: 0.7693
Epoch 47/100
1634/1634 [==============================] - 2s 1ms/step - loss: 0.4728 - acc: 0.7852
Epoch 48/100
1634/1634 [==============================] - 2s 1ms/step - loss: 0.5081 - acc: 0.7576
Epoch 49/100
1634/1634 [==============================] - 2s 1ms/step - loss: 0.4642 - acc: 0.7938
Epoch 50/100
1634/1634 [==============================] - 2s 1ms/step - loss: 0.4766 - acc: 0.7778
Epoch 51/100
1634/1634 [==============================] - 2s 1ms/step - loss: 0.4794 - acc: 0.7803
Epoch 52/100
1634/1634 [==============================] - 2s 1ms/step - loss: 0.4614 - acc: 0.7950
Epoch 53/100
1634/1634 [==============================] - 2s 1ms/step - loss: 0.4500 - acc: 0.8097
Epoch 54/100
1634/1634 [==============================] - 2s 1ms/step - loss: 0.4753 - acc: 0.7895
Epoch 55/100
1634/1634 

3267/3267 [==============================] - 4s 1ms/step - loss: 0.4608 - acc: 0.8032
Epoch 28/100
3267/3267 [==============================] - 4s 1ms/step - loss: 0.4605 - acc: 0.8038
Epoch 29/100
3267/3267 [==============================] - 4s 1ms/step - loss: 0.4795 - acc: 0.7903
Epoch 30/100
3267/3267 [==============================] - 4s 1ms/step - loss: 0.4502 - acc: 0.8124
Epoch 31/100
3267/3267 [==============================] - 4s 1ms/step - loss: 0.4650 - acc: 0.8059
Epoch 32/100
3267/3267 [==============================] - 4s 1ms/step - loss: 0.4660 - acc: 0.8013
Epoch 33/100
3267/3267 [==============================] - 4s 1ms/step - loss: 0.4463 - acc: 0.8194
Epoch 34/100
3267/3267 [==============================] - 4s 1ms/step - loss: 0.4557 - acc: 0.8087
Epoch 35/100
3267/3267 [==============================] - 4s 1ms/step - loss: 0.4449 - acc: 0.8185
Epoch 36/100
3267/3267 [==============================] - 4s 1ms/step - loss: 0.4619 - acc: 0.8023
Epoch 37/100
3267/3267 

...

3267/3267 [==============================] - 5s 1ms/step - loss: 0.5918 - acc: 0.7224
Epoch 160/250
3267/3267 [==============================] - 5s 1ms/step - loss: 0.5914 - acc: 0.7224
Epoch 161/250
3267/3267 [==============================] - 5s 1ms/step - loss: 0.5913 - acc: 0.7224
Epoch 162/250
3267/3267 [==============================] - 5s 1ms/step - loss: 0.5912 - acc: 0.7224
Epoch 163/250
3267/3267 [==============================] - 5s 1ms/step - loss: 0.5913 - acc: 0.7224
Epoch 164/250
3267/3267 [==============================] - 5s 1ms/step - loss: 0.5914 - acc: 0.7224
Epoch 165/250
3267/3267 [==============================] - 5s 1ms/step - loss: 0.5916 - acc: 0.7224
Epoch 166/250
3267/3267 [==============================] - 5s 1ms/step - loss: 0.5910 - acc: 0.7224
Epoch 167/250
3267/3267 [==============================] - 5s 1ms/step - loss: 0.5918 - acc: 0.7224
Epoch 168/250
3267/3267 [==============================] - 5s 1ms/step - loss: 0.5918 - acc: 0.7224
Epoch 169/250


3267/3267 [==============================] - 5s 1ms/step - loss: 0.5914 - acc: 0.7224
Epoch 242/250
3267/3267 [==============================] - 5s 1ms/step - loss: 0.5910 - acc: 0.7224
Epoch 243/250
3267/3267 [==============================] - 5s 1ms/step - loss: 0.5911 - acc: 0.7224
Epoch 244/250
3267/3267 [==============================] - 5s 1ms/step - loss: 0.5910 - acc: 0.7224
Epoch 245/250
3267/3267 [==============================] - 5s 1ms/step - loss: 0.5911 - acc: 0.7224
Epoch 246/250
3267/3267 [==============================] - 5s 1ms/step - loss: 0.5909 - acc: 0.7224
Epoch 247/250
3267/3267 [==============================] - 5s 1ms/step - loss: 0.5916 - acc: 0.7224
Epoch 248/250
3267/3267 [==============================] - 5s 1ms/step - loss: 0.5912 - acc: 0.7224
Epoch 249/250
3267/3267 [==============================] - 5s 1ms/step - loss: 0.5910 - acc: 0.7224
Epoch 250/250
1633/1633 [==============================] - 6s 4ms/step
Epoch 1/250
4900/4900 [====================

4900/4900 [==============================] - 5s 943us/step - loss: 0.4238 - acc: 0.8029
Epoch 72/250
4900/4900 [==============================] - 5s 939us/step - loss: 0.4265 - acc: 0.7998
Epoch 73/250
4900/4900 [==============================] - 5s 947us/step - loss: 0.4260 - acc: 0.8002
Epoch 74/250
4900/4900 [==============================] - 5s 936us/step - loss: 0.4227 - acc: 0.8084
Epoch 75/250
4900/4900 [==============================] - 5s 945us/step - loss: 0.4245 - acc: 0.8031
Epoch 76/250
4900/4900 [==============================] - 5s 940us/step - loss: 0.4284 - acc: 0.8061
Epoch 77/250
4900/4900 [==============================] - 5s 938us/step - loss: 0.4227 - acc: 0.8035
Epoch 78/250
4900/4900 [==============================] - 5s 947us/step - loss: 0.4193 - acc: 0.8047
Epoch 79/250
4900/4900 [==============================] - 5s 938us/step - loss: 0.4233 - acc: 0.7988
Epoch 80/250
4900/4900 [==============================] - 5s 940us/step - loss: 0.4209 - acc: 0.8043
Epo

4900/4900 [==============================] - 5s 947us/step - loss: 0.4187 - acc: 0.8039
Epoch 152/250
4900/4900 [==============================] - 5s 945us/step - loss: 0.4148 - acc: 0.8078
Epoch 153/250
4900/4900 [==============================] - 5s 942us/step - loss: 0.4155 - acc: 0.8069
Epoch 154/250
4900/4900 [==============================] - 5s 944us/step - loss: 0.4181 - acc: 0.7998
Epoch 155/250
4900/4900 [==============================] - 5s 939us/step - loss: 0.4157 - acc: 0.8080
Epoch 156/250
4900/4900 [==============================] - 5s 942us/step - loss: 0.4203 - acc: 0.8049
Epoch 157/250
4900/4900 [==============================] - 5s 941us/step - loss: 0.4193 - acc: 0.8020
Epoch 158/250
4900/4900 [==============================] - 5s 942us/step - loss: 0.4162 - acc: 0.8018
Epoch 159/250
4900/4900 [==============================] - 5s 946us/step - loss: 0.4152 - acc: 0.8031
Epoch 160/250
4900/4900 [==============================] - 5s 943us/step - loss: 0.4182 - acc: 0

4900/4900 [==============================] - 5s 945us/step - loss: 0.4146 - acc: 0.8022
Epoch 232/250
4900/4900 [==============================] - 5s 941us/step - loss: 0.4149 - acc: 0.8045
Epoch 233/250
4900/4900 [==============================] - 5s 944us/step - loss: 0.4195 - acc: 0.8035
Epoch 234/250
4900/4900 [==============================] - 5s 941us/step - loss: 0.4119 - acc: 0.8071
Epoch 235/250
4900/4900 [==============================] - 5s 947us/step - loss: 0.4159 - acc: 0.8090
Epoch 236/250
4900/4900 [==============================] - 5s 941us/step - loss: 0.4130 - acc: 0.8055
Epoch 237/250
4900/4900 [==============================] - 5s 938us/step - loss: 0.4119 - acc: 0.8053
Epoch 238/250
4900/4900 [==============================] - 5s 942us/step - loss: 0.4131 - acc: 0.8102
Epoch 239/250
4900/4900 [==============================] - 5s 938us/step - loss: 0.4151 - acc: 0.8086
Epoch 240/250
4900/4900 [==============================] - 5s 939us/step - loss: 0.4146 - acc: 0

In [11]:
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: 0.248283 using {'epochs': 100, 'hidden_unit': 48, 'l2_weight': 0.001, 'n_layer': 4}
0.070421 (0.110061) with: {'epochs': 100, 'hidden_unit': 48, 'l2_weight': 0.001, 'n_layer': 3}
0.248283 (0.138217) with: {'epochs': 100, 'hidden_unit': 48, 'l2_weight': 0.001, 'n_layer': 4}
0.056762 (0.056762) with: {'epochs': 100, 'hidden_unit': 48, 'l2_weight': 0.01, 'n_layer': 3}
0.000000 (0.000000) with: {'epochs': 100, 'hidden_unit': 48, 'l2_weight': 0.01, 'n_layer': 4}
0.092426 (0.053107) with: {'epochs': 100, 'hidden_unit': 64, 'l2_weight': 0.001, 'n_layer': 3}
0.110435 (0.047167) with: {'epochs': 100, 'hidden_unit': 64, 'l2_weight': 0.001, 'n_layer': 4}
0.095785 (0.095785) with: {'epochs': 100, 'hidden_unit': 64, 'l2_weight': 0.01, 'n_layer': 3}
0.000000 (0.000000) with: {'epochs': 100, 'hidden_unit': 64, 'l2_weight': 0.01, 'n_layer': 4}
0.074381 (0.114191) with: {'epochs': 250, 'hidden_unit': 48, 'l2_weight': 0.001, 'n_layer': 3}
0.086612 (0.119223) with: {'epochs': 250, 'hidden_unit': 48

## Best LSTM params
best params = {'epochs': 236, 'n_layer': 3, 'hidden_unti': 64, 'l2_weight': 0}


## We have 2984 positive and 984 negtive cases. 

In [ ]:
#Make predictions for train set   
print(collections.Counter(train_Y))
print_report_for_binary_classfier(train_Y, lstm_clf.predict(train_X))

In [ ]:
#Make predictions from dev set
print(collections.Counter(val_Y))
print_report_for_binary_classfier(val_Y, lstm_clf.predict((val_X)))

In [32]:
#Make predictions for test set
print(collections.Counter(test_Y))
print_report_for_binary_classfier(test_Y, lstm_clf.predict((test_X)))

Counter({0.0: 811, 1.0: 30})
841/841 [==============================] - 6s 7ms/step
F1 score: 0.000000
precision score: 0.000000
recall score: 0.000000
accuracy score: 0.964328
matthews_corrcoef: 0.000000

Confusion matrix:
[[811   0]
 [ 30   0]]


/home/kevin/anaconda2/envs/py36/lib/python3.6/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/kevin/anaconda2/envs/py36/lib/python3.6/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


# Save the best model

In [26]:
# from keras.models import load_model

# model.save(params.BEST_LSTM_MODEL_PATH)  # creates a HDF5 file 'my_model.h5'